BAM학습

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os

def load_and_preprocess_images(folder_path, image_size):
    images = []
    for filename in os.listdir(folder_path)[:40]:  # 처음 40개의 이미지만 사용
        if filename.endswith(('.png', '.jpg', '.jpeg')):
            img_path = os.path.join(folder_path, filename)
            img = Image.open(img_path).convert('L')  # 그레이스케일로 변환
            img = img.resize((image_size, image_size))
            img_array = np.array(img)
            img_binary = np.where(img_array > 128, 1, -1)  # 이진화
            images.append(img_binary.flatten())  # 1차원 배열로 변환
    return images

def add_noise(image, noise_factor=0.2):
    noisy_img = image.copy()
    noise = np.random.rand(*image.shape) < noise_factor
    noisy_img[noise] *= -1
    return noisy_img

In [41]:
class BAM:
    def __init__(self, size_x, size_y):
        self.size_x = size_x
        self.size_y = size_y
        self.W = np.zeros((size_x, size_y))

    def train(self, patterns_x, patterns_y):
        for x, y in zip(patterns_x, patterns_y):
            self.W += np.outer(x, y)  # Hebbian learning
        self.W /= len(patterns_x)

    def recall_x(self, y, max_iterations=50):
        for _ in range(max_iterations):
            x = np.sign(self.W @ y)
            y_new = np.sign(self.W.T @ x)
            if np.array_equal(y, y_new):
                break
            y = y_new
        return x

    def recall_y(self, x, max_iterations=50):
        for _ in range(max_iterations):
            y = np.sign(self.W.T @ x)
            x_new = np.sign(self.W @ y)
            if np.array_equal(x, x_new):
                break
            x = x_new
        return y

In [1]:
# 이미지 로드 및 전처리
image_size = 32
folder_path = "data"
images = load_and_preprocess_images(folder_path, image_size)

# BAM 생성 및 학습
bam = BAM(image_size * image_size, image_size * image_size)
bam.train(images, images)  # 자기 연관 학습 (self-association)

# 모든 학습된 이미지에 대해 테스트
plt.figure(figsize=(15, 5 * len(images)))
for i, original_image in enumerate(images):
    noisy_image = add_noise(original_image)
    recovered_image = bam.recall_x(noisy_image)

    plt.subplot(len(images), 3, i*3 + 1)
    plt.imshow(original_image.reshape(image_size, image_size), cmap='gray')
    plt.title(f'Original Image {i+1}')
    plt.axis('off')

    plt.subplot(len(images), 3, i*3 + 2)
    plt.imshow(noisy_image.reshape(image_size, image_size), cmap='gray')
    plt.title(f'Noisy Image {i+1}')
    plt.axis('off')

    plt.subplot(len(images), 3, i*3 + 3)
    plt.imshow(recovered_image.reshape(image_size, image_size), cmap='gray')
    plt.title(f'Recovered Image {i+1}')
    plt.axis('off')

plt.tight_layout()
plt.show()

# 학습되지 않은 이미지로 테스트
new_image = load_and_preprocess_images(folder_path, image_size, num_images=6)[-1]
noisy_new_image = add_noise(new_image)
recovered_new_image = bam.recall_x(noisy_new_image)

plt.figure(figsize=(15, 5))
plt.subplot(131)
plt.imshow(new_image.reshape(image_size, image_size), cmap='gray')
plt.title('New Image (Not Learned)')
plt.axis('off')

plt.subplot(132)
plt.imshow(noisy_new_image.reshape(image_size, image_size), cmap='gray')
plt.title('Noisy New Image')
plt.axis('off')

plt.subplot(133)
plt.imshow(recovered_new_image.reshape(image_size, image_size), cmap='gray')
plt.title('BAM Output for New Image')
plt.axis('off')

plt.tight_layout()
plt.show()

NameError: name 'load_and_preprocess_images' is not defined